In [1]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 1

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\alex\code\machine-learning\env36\lib\site-packages\tensorflow\python\f

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from datetime import datetime
from scipy.stats import uniform, randint

In [3]:
df = pd.read_csv("train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
onehot = OneHotEncoder(handle_unknown='ignore')
onehot.fit(df[["label"]])
cat_list = np.concatenate(onehot.categories_)
df[cat_list] = pd.DataFrame(onehot.transform(df[["label"]]).toarray(), index=df.index)

In [5]:
X = df.drop(columns=cat_list)
X = X.drop(columns=["label"])
X = X / 255
y = df[cat_list]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(33600, 784) (33600, 10)
(8400, 784) (8400, 10)


In [6]:
class LogUniform:
    def __init__(self, low, high):
        self.low = low
        self.high = high
    
    def rvs(self, *args, **kwds):
        return 10 ** np.random.uniform(np.log10(self.low), np.log10(self.high))

In [7]:
def create_model(learning_rate, units, l2reg=0, hidden_layers=1):
    # cleanup
    K.clear_session()
    
    model = Sequential()
    model.add(Dense(units=units, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(l2reg)))
    for i in range(hidden_layers - 1):
        model.add(Dense(units=units, activation='relu', kernel_regularizer=l2(l2reg)))
    
    model.add(Dense(units=10, activation="softmax", kernel_regularizer=l2(l2reg)))
    model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate),
              metrics=['accuracy'])
    
    return model


# model = KerasClassifier(build_fn=create_model, epochs=50)
# param_dist = {"learning_rate": LogUniform(0.0001, 1),
#               "l2reg": LogUniform(0.0001, 1),
#               "units": randint(20, 200),
#               "batch_size": [16, 32, 64, 128],
#               #"hidden_layers": [1]
#              }
# clf = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_jobs=1, cv=5, n_iter=20, verbose=100)
# clf.fit(X_train, y_train)


l2reg = 0.0003
clf = Sequential()
clf.add(Dense(units=100, activation='relu', input_dim=X_train.shape[1], kernel_regularizer=l2(l2reg)))
clf.add(Dense(units=10, activation="softmax", kernel_regularizer=l2(l2reg)))
clf.compile(loss='categorical_crossentropy',
          optimizer=Adam(0.0009),
          metrics=['accuracy'])

clf.fit(X_train, y_train, epochs=20, batch_size=32)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
33600/33600 [==============================] - 4s 106us/step - loss: 0.4003 - acc: 0.9020
Epoch 2/20
33600/33600 [==============================] - 3s 99us/step - loss: 0.2311 - acc: 0.9512
Epoch 3/20
33600/33600 [==============================] - 3s 96us/step - loss: 0.1957 - acc: 0.9628
Epoch 4/20
33600/33600 [==============================] - 3s 97us/step - loss: 0.1773 - acc: 0.9689
Epoch 5/20
33600/33600 [==============================] - 4s 105us/step - loss: 0.1655 - acc: 0.9725
Epoch 6/20
33600/33600 [==============================] - 3s 98us/step - loss: 0.1580 - acc: 0.9752
Epoch 7/20
33600/33600 [==============================] - 3s 98us/step - loss: 0.1535 - acc: 0.9765
Epoch 8/20
15680/33600 [=============>................] - ETA: 1s - loss: 0.1452 - acc: 0.9802

KeyboardInterrupt: 

In [ ]:
# print(clf.best_params_)
# train_score = clf.score(X_train, y_train)
# test_score = clf.score(X_test, y_test)
# print(train_score, test_score)

train_loss_and_metrics = clf.evaluate(X_train, y_train)
test_loss_and_metrics = clf.evaluate(X_test, y_test)
print(train_loss_and_metrics, test_loss_and_metrics)

In [ ]:
df_test = pd.read_csv("test.csv")
df_test = df_test / 255
predictions = clf.predict(df_test)
predictions = np.squeeze(onehot.inverse_transform(predictions))
df_submit = pd.DataFrame({'ImageId': list(range(1, 28001)), 'Label': predictions})
df_submit.to_csv('submission.csv', index=False)